In [1]:
# bfs concurrency
import requests
from bs4 import BeautifulSoup
from collections import deque
import re
import concurrent.futures
from urllib import request

def bfs(url):
    visited = set([url])
    dq = deque([[url, "", 0]])
    max_depth = 3
    max_breadth = 5
    string = ""
    return_string =""
    while dq:
        base, path, depth = dq.popleft()
        if depth < max_depth:
            try:
                soup = BeautifulSoup(requests.get(base + path).text, "html.parser")
                links = soup.find_all("a")[:max_breadth]
                for link in links:
                    href = link.get("href")
                    if href not in visited:
                        visited.add(href)
                        string += ("  " * depth + f"at depth {depth}: {href}\n")
                        return_string += (href)+' '

                        if href.startswith("http") or href.startswith("https"):
                            dq.append([href, "", depth + 1])
                        else:
                            dq.append([base, href, depth + 1])
            except:
                pass

    filename =""
    if "https://" in url:
        filename = re.sub(r"https://","",url)
        filename = re.sub(r".com","",filename)
    elif "http://" in url:
        filename = re.sub(r"http://","",url)
        filename = re.sub(r".com","",filename)

    file  = open("crawled_links/"+filename+".txt","w")
    file.write(string)
    file.close()
    
    return return_string

In [2]:
frontier = ["http://toscrape.com", "https://soundcloud.com", "http://reddit.com", "https://fc2.com"]

link_string_arr = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(bfs, i) for i in frontier]
    print("started")
    link_string_arr = [f.result() for f in futures]
print("files made!")

started
files made!


In [3]:
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd


stop_words = set(STOP_WORDS)
stop_words = stop_words.union({'.',',','\'','\"','?','{','}','[',']','<','>','(',')','!'})

stemmer = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

base = 'https://en.wikipedia.org/wiki/'
links = ['Web_mining','Data_mining']
terms_searched = []
# assuming these are the links crawled by the crawler

for i in links:
    url = base + i
    html = request.urlopen(url).read().decode('utf8')
    data = list(set(word_tokenize(BeautifulSoup(html, 'html.parser').get_text())).difference(stop_words))
    # data = set(word_tokenize((BeautifulSoup(requests.urlopen(url).read().decode('utf8')), 'html.parser').get_text())).difference(stop_words)
    stemmed = list(set([stemmer.stem(w) for w in data]))
    # lemmatized = list(set([lemmatizer.lemmatize(w) for w in data]))
    terms_searched.append(stemmed)

terms_data = []
for i in terms_searched:
    for j in i:
        terms_data.append(j)

terms_data.sort()

lister = []
for i in terms_data:
    bigger = ''
    if i in terms_searched[0]:
        indices = []
        for j in range(len(terms_searched[0])):
            if terms_searched[0][j] == i:
                indices.append(j)
        bigger+= 'Doc 1  count: ' + str(len(indices)) + str(indices) + ' '
    if i in terms_searched[1]:
        indices = []
        for j in range(len(terms_searched[1])):
            if terms_searched[1][j] == i:
                indices.append(j)
        bigger+= 'Doc 2  count: ' + str(len(indices)) + str(indices) + ' '
    lister.append(bigger)

pd.DataFrame({
    "Terms" : terms_data,
    "index" : lister
})

,Terms,index
0,%,Doc 2 count: 1[155]
1,&,Doc 1 count: 1[81] Doc 2 count: 1[848]
2,&,Doc 1 count: 1[81] Doc 2 count: 1[848]
3,'',Doc 1 count: 1[909] Doc 2 count: 1[1615]
4,'',Doc 1 count: 1[909] Doc 2 count: 1[1615]
...,...,...
2530,“,Doc 1 count: 1[459] Doc 2 count: 1[589]
2531,”,Doc 1 count: 1[843] Doc 2 count: 1[779]
2532,”,Doc 1 count: 1[843] Doc 2 count: 1[779]
2533,•,Doc 2 count: 1[1597]


In [4]:
import math

def unary(x):
    return '0'*(x-1)+'1'

def encoder(x, b):
    q = math.floor(x/b)
    q_encode = unary(q+1)
    i = math.floor(math.log2(b))
    d = int(2**(i+1))-b
    r = x%b
    rem = ""
    if r < d:
        rem = bin(r)[2:]
        l = len(rem)
        if l < i:
            rem = '0'*(i - l)+rem
    else:
        rem = bin(r + d)[2:]
        l = len(rem)
        if l < i+1:
            rem = '0'*(i+1-l) + rem
    return q_encode+rem

print(encoder(74, 16))
print(encoder(50, 11))

000011010
000011011


In [5]:
def decode(n, M):
    q=len(n.split('1')[0])
    b=math.floor(math.log2(M))
    k = 2**(b+1)-M
    r = int(n[q+1:q+1+b],2)
    if r>=k:
        r = int(n[q+1:q+1+b+1],2)
        r=r-k
    x=q*M+r
    return x

print(decode("1111111110010001101", 10))

9


In [6]:
# Q4
from functools import reduce
import pandas as pd

def extract(url):
    html = request.urlopen(url).read().decode('utf8')
    string = BeautifulSoup(html, 'html.parser').get_text().lower()
    return (list(word_tokenize(string)), string)

def store(name, ls):
    with open(name,'w') as f:
        for item in list(ls):
            try:
                f.write('%s\n'%item)
            except:
                pass
def term_maker(ls, filenames):
    terms = []
    index = 0
    for i in ls:
        st = set(i).difference(stop_words)
        store('extracted_content/'+filenames[index]+'.txt', list(st))
        index += 1
        terms.append(list(st))
    terms = reduce(lambda z,y : z+y, terms)
    return terms

def counter(tokens, terms):
    ls = []
    for i in terms:
        ls.append(tokens.count(i))
    return ls

base = 'https://en.wikipedia.org/wiki/'
links = ['Web_mining','Data_mining','Artificial_intelligence','Machine_learning','Natural_language_processing','Text_mining']
token_list = []
for i in links:
    token_list.append(extract(base+i)[0])
terms = term_maker(token_list, links)
bagOfWords = []
for i in token_list:
    bagOfWords.append(counter(i,terms))

pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : bagOfWords[0],     
    "Doc 2" : bagOfWords[1],     
    "Doc 3" : bagOfWords[2],     
    "Doc 4" : bagOfWords[3],     
    "Doc 5" : bagOfWords[4],     
    "Doc 6" : bagOfWords[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,11,2,4,16,9,1,3
1,sophisticated,1,0,2,0,0,0
2,links,6,4,5,4,2,4
3,doi:10.1145/360402.360406,1,0,0,0,0,0
4,demonstrated,1,0,3,0,0,2
...,...,...,...,...,...,...,...
15125,management,3,9,11,4,3,7
15126,linked,0,0,0,0,0,1
15127,lab,0,0,0,1,0,1
15128,lemmatisation,0,0,0,0,1,1


In [7]:
def total(ls):
    count=0
    for i in ls:
        if i != 0:
            count+=1
    return count

TF = [[number/total(i) for number in i] for i in bagOfWords]
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF[0],     
    "Doc 2" : TF[1],     
    "Doc 3" : TF[2],     
    "Doc 4" : TF[3],     
    "Doc 5" : TF[4],     
    "Doc 6" : TF[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,11,0.000576,0.000706,0.001458,0.001161,0.000186,0.000581
1,sophisticated,0.000288,0.000000,0.000182,0.000000,0.000000,0.000000
2,links,0.001728,0.000706,0.000456,0.000516,0.000372,0.000775
3,doi:10.1145/360402.360406,0.000288,0.000000,0.000000,0.000000,0.000000,0.000000
4,demonstrated,0.000288,0.000000,0.000273,0.000000,0.000000,0.000387
...,...,...,...,...,...,...,...
15125,management,0.000864,0.001589,0.001002,0.000516,0.000559,0.001356
15126,linked,0.000000,0.000000,0.000000,0.000000,0.000000,0.000194
15127,lab,0.000000,0.000000,0.000000,0.000129,0.000000,0.000194
15128,lemmatisation,0.000000,0.000000,0.000000,0.000000,0.000186,0.000194


In [8]:
import math
def IDF_helper(ls):
    total = []
    length = len(ls[0])
    for i in range(length):
        count = 0
        for j in range(len(ls)):
            value = ls[j][i]
            if value != 0:
                count += 1
        total.append(count)
    return total

def IDF_calc():
    helper = IDF_helper(bagOfWords)
    no_of_doc = 6
    IDF_ls = []
    for i in helper:
        IDF_ls.append(math.log((1+no_of_doc)/i))
    return IDF_ls
pd.DataFrame({
    "Terms" : list(terms),
    "IDF value" : IDF_calc()
})

,Terms,IDF value
0,11,0.154151
1,sophisticated,1.252763
2,links,0.154151
3,doi:10.1145/360402.360406,1.945910
4,demonstrated,0.847298
...,...,...
15125,management,0.154151
15126,linked,1.945910
15127,lab,1.252763
15128,lemmatisation,1.252763


In [9]:
def TF_IDF_calc(ls):
    IDF = IDF_calc()
    for i in range(len(ls[0])):
        for j in range(6):
                ls[j][i]*=IDF[i]
    return ls
TF_IDF = TF_IDF_calc(TF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF_IDF[0],
    "Doc 2" : TF_IDF[1],
    "Doc 3" : TF_IDF[2],
    "Doc 4" : TF_IDF[3],
    "Doc 5" : TF_IDF[4],
    "Doc 6" : TF_IDF[5]
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,11,0.000089,0.000109,0.000225,0.000179,0.000029,0.000090
1,sophisticated,0.000361,0.000000,0.000228,0.000000,0.000000,0.000000
2,links,0.000266,0.000109,0.000070,0.000080,0.000057,0.000119
3,doi:10.1145/360402.360406,0.000560,0.000000,0.000000,0.000000,0.000000,0.000000
4,demonstrated,0.000244,0.000000,0.000232,0.000000,0.000000,0.000328
...,...,...,...,...,...,...,...
15125,management,0.000133,0.000245,0.000154,0.000080,0.000086,0.000209
15126,linked,0.000000,0.000000,0.000000,0.000000,0.000000,0.000377
15127,lab,0.000000,0.000000,0.000000,0.000162,0.000000,0.000243
15128,lemmatisation,0.000000,0.000000,0.000000,0.000000,0.000233,0.000243


In [10]:
def query_TF_IDF(string):
    ref_tokens = list(word_tokenize(string.lower()))
    tokens = list(set(word_tokenize(string.lower())).difference(stop_words))
    Tf_query = []
    for i in tokens:
        Tf_query.append(1/ref_tokens.count(i))
    ref_IDF = IDF_calc()
    TF_IDF = []
    for i in range(len(tokens)):
        if tokens[i] in terms:
            index = terms.index(tokens[i])
            value = ref_IDF[index]*Tf_query[i]
            TF_IDF.append(value)
        else:
            TF_IDF.append(0)
    return (tokens,TF_IDF)

string = "I want communications based on machine learning"
Q_TF_IDF = query_TF_IDF(string)
pd.DataFrame({
    "Terms" : Q_TF_IDF[0],
    "TF_IDF" : Q_TF_IDF[1]
})

,Terms,TF_IDF
0,machine,0.154151
1,based,0.154151
2,learning,0.154151
3,communications,1.252763
4,want,1.945910


In [11]:
def normalization_Document(TF_IDF):
    no_of_docs = len(TF_IDF)
    for i in range(no_of_docs):
        divider = 0
        for j in TF_IDF[i]:
            divider += j**2
        divider = math.sqrt(divider)
        TF_IDF[i] = [k/divider for k in TF_IDF[i]]
    return TF_IDF

normalized = normalization_Document(TF_IDF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : normalized[0],
    "Doc 2" : normalized[1],
    "Doc 3" : normalized[2],
    "Doc 4" : normalized[3],
    "Doc 5" : normalized[4],
    "Doc 6" : normalized[5],
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,11,0.002804,0.003442,0.002965,0.004997,0.000866,0.003176
1,sophisticated,0.011395,0.000000,0.003012,0.000000,0.000000,0.000000
2,links,0.008413,0.003442,0.000927,0.002221,0.001732,0.004235
3,doi:10.1145/360402.360406,0.017700,0.000000,0.000000,0.000000,0.000000,0.000000
4,demonstrated,0.007707,0.000000,0.003056,0.000000,0.000000,0.011639
...,...,...,...,...,...,...,...
15125,management,0.004206,0.007745,0.002038,0.002221,0.002598,0.007411
15126,linked,0.000000,0.000000,0.000000,0.000000,0.000000,0.013365
15127,lab,0.000000,0.000000,0.000000,0.004512,0.000000,0.008604
15128,lemmatisation,0.000000,0.000000,0.000000,0.000000,0.007038,0.008604


In [12]:
def normalization_Query(string):
    TF_IDF = query_TF_IDF(string)[1]
    divider = 0
    for i in TF_IDF:
        divider += i**2
    TF_IDF = [i/divider for i in TF_IDF]
    return TF_IDF

pd.DataFrame({
    "Terms" : query_TF_IDF(string)[0],
    "TF_IDF" : normalization_Query(string)
})

,Terms,TF_IDF
0,machine,0.028403
1,based,0.028403
2,learning,0.028403
3,communications,0.230828
4,want,0.358543


In [13]:
def cosine_dist():
    TF_IDF_doc = normalization_Document(TF_IDF)
    TF_IDF_query = normalization_Query(string)
    tokens = Q_TF_IDF[0]
    index_ls = []
    for i in tokens:
        index_ls.append(terms.index(i))
    
    cosine_dict = {}
    for i in range(len(TF_IDF_doc)):
        cosine_dist = 0
        index = 0
        for j in index_ls:
            cosine_dist += TF_IDF_doc[i][j]* TF_IDF_query[index]
            index+=1
        cosine_dict['Doc'+str(i+1)] = cosine_dist
    return cosine_dict

pd.DataFrame(list(cosine_dist().items()), columns=['Documents', 'Cosine Dist'])

,Documents,Cosine Dist
0,Doc1,0.005778
1,Doc2,0.002249
2,Doc3,0.003736
3,Doc4,0.007917
4,Doc5,0.001648
5,Doc6,0.000662


In [14]:
def euclidean_dist():
    TF_IDF_doc = normalization_Document(TF_IDF)
    TF_IDF_query = normalization_Query(string)
    tokens = Q_TF_IDF[0]
    index_ls = []
    for i in tokens:
        index_ls.append(terms.index(i))
    
    euclidean_dict = {}
    for i in range(len(TF_IDF_doc)):
        euclidean_dist = 0
        index = 0
        for j in index_ls:
            euclidean_dist += (TF_IDF_doc[i][j] - TF_IDF_query[index])**2
            index+=1
        euclidean_dist = math.sqrt(euclidean_dist)
        euclidean_dict['Doc'+str(i+1)] = euclidean_dist
    return euclidean_dict

pd.DataFrame(list(euclidean_dist().items()), columns=['Documents', 'Euclidean Dist'])

,Documents,Euclidean Dist
0,Doc1,0.416352
1,Doc2,0.427644
2,Doc3,0.421194
3,Doc4,0.456539
4,Doc5,0.426861
5,Doc6,0.427928


In [27]:
sorted_dict = {k: v for k,v in sorted(cosine_dist().items(), key=lambda item: item[1])}

pd.DataFrame(list(sorted_dict.items())[::-1])

,0,1
0,Doc4,0.007917
1,Doc1,0.005778
2,Doc3,0.003736
3,Doc2,0.002249
4,Doc5,0.001648
5,Doc6,0.000662


In [28]:
sorted_dict = {k: v for k,v in sorted(euclidean_dist().items(), key=lambda item: item[1])}

pd.DataFrame(list(sorted_dict.items()))

,0,1
0,Doc1,0.416352
1,Doc3,0.421194
2,Doc5,0.426861
3,Doc2,0.427644
4,Doc6,0.427928
5,Doc4,0.456539


In [16]:
def doc_similarity():
    TF_IDF_doc = normalization_Document(TF_IDF)
    len_doc = len(TF_IDF_doc[0])
    cosine_sim = []
    for i in range(1, len(TF_IDF_doc)):
        temp = 0
        for j in range(len_doc):
            temp += TF_IDF_doc[0][j] * TF_IDF_doc[i][j]
        cosine_sim.append(temp)
    return cosine_sim

ls = ["Doc1 and Doc2", "Doc1 and Doc3", "Doc1 and Doc4", "Doc1 and Doc5", "Doc1 and Doc6"]

pd.DataFrame({
    "Similarity between" : ls,
    "Cosine value" : doc_similarity()
})

,Similarity between,Cosine value
0,Doc1 and Doc2,0.369324
1,Doc1 and Doc3,0.135916
2,Doc1 and Doc4,0.182782
3,Doc1 and Doc5,0.145552
4,Doc1 and Doc6,0.277352


In [21]:
def jaccardSimilarity(word1,word2) : 
    common_words = len(set(set(list(word1))&set(list(word2))))
    total_words = len(set(list(word1)+list(word2)))
    return common_words/total_words

class TrieNode():
    def __init__(self): 
        self.children = {} 
        self.last = False

class Trie(): 
    
    def __init__(self): 
        self.root = TrieNode() 
        self.word_list = [] 
  
    def formTrie(self, keys): 
        for key in keys: 
            self.insert(key) 
  
    def insert(self, key): 
        node = self.root 
  
        for a in list(key): 
            if not node.children.get(a): 
                node.children[a] = TrieNode() 
  
            node = node.children[a] 
  
        node.last = True
  
    def suggestionsRec(self, node, word):  #recursive
        if node.last: 
            self.word_list.append(word) 
  
        for a,n in node.children.items(): 
            self.suggestionsRec(n, word + a) 
  
    def PredictiveTyping(self, key): 
        node = self.root
        temp_word = '' 
  
        for a in list(key): 
            if not node.children.get(a):
                break
  
            temp_word += a 
            node = node.children[a] 
  
        self.suggestionsRec(node, temp_word) 
        return self.word_list
    
    def autoCorrect(self,key) :
        allwords = self.PredictiveTyping("")
        highest_jacsims = 0
        highest_jacsims_word = allwords[0]
        for word in allwords :
            temp_sim = jaccardSimilarity(word, key)
            if temp_sim > highest_jacsims :
                highest_jacsims = temp_sim
                highest_jacsims_word = word
        return highest_jacsims_word
    
    def printTrie(self) :
        print(str(self.root.children) + "\n")
  
# Driver Code 
keys = ["hello", "dog", "hell", "cat", "a",  
        "hel", "help", "helps", "helping"] # keys to form the trie structure. 
key = "hel" 
  

#for artificial intelligence link - trie structure

print("\n\nArtificial Intelligence Wikipedia\n\n")
raw1 = extract("https://en.wikipedia.org/wiki/Artificial_intelligence")[1]

url1_words = re.findall(r'[a-zA-z]+', raw1)

STOP_WORDS.update(['.',',', "'",'"' , "?", "[","]","(",")","{","}","<",">","!"])

url1_nostopwords = [x for x in url1_words if x not in STOP_WORDS]

t1 = Trie()  
t1.formTrie(url1_nostopwords) 
# t1.printTrie() 
predTemp = input()
print("predictive typing " + predTemp + " : " + str(t1.PredictiveTyping(predTemp)) + "\n")
autoTemp = input()
print("predicted by autocorrect : "+ str(t1.autoCorrect(autoTemp)))


print("\n\nMachine Learning Wikipedia\n\n")

raw2 = extract("https://en.wikipedia.org/wiki/Machine_learning")[1]
url2_words = re.findall(r'[a-zA-z]+', raw2)

#adding manual stop words to the STOP_WORDS set
STOP_WORDS.update(['.',',', "'",'"' , "?", "[","]","(",")","{","}","<",">","!"])

#removing stop words from url
url2_nostopwords = [x for x in url2_words if x not in STOP_WORDS]

#creating the trie structure
t2 = Trie()  
t2.formTrie(url2_nostopwords) 
# t2.printTrie() 
predTemp = input()
print("predictive typing " + predTemp + " : " + str(t2.PredictiveTyping(predTemp)) + "\n")
autoTemp = input()
print("predicted by autocorrect : "+ str(t1.autoCorrect(autoTemp)))




Artificial Intelligence Wikipedia


predictive typing artifi : ['artificial', 'artificialintelligence', 'artificial_intelligence']

predicted by autocorrect : intelligent


Machine Learning Wikipedia


predictive typing machi : ['machine', 'machines', 'machines[edit]', 'machinery', 'machine_learning']

predicted by autocorrect : learn


In [22]:
t1.printTrie()

{'a': <__main__.TrieNode object at 0x00000273B4CEF160>, 'i': <__main__.TrieNode object at 0x00000273B4CEF198>, 'w': <__main__.TrieNode object at 0x00000273B4CEF048>, 'f': <__main__.TrieNode object at 0x00000273B4CEF7B8>, 'e': <__main__.TrieNode object at 0x00000273B4CEF860>, 'j': <__main__.TrieNode object at 0x00000273B4CEFC88>, 'n': <__main__.TrieNode object at 0x00000273B4CEFBA8>, 's': <__main__.TrieNode object at 0x00000273B5F6B940>, 'r': <__main__.TrieNode object at 0x00000273B5F6BC88>, 'u': <__main__.TrieNode object at 0x00000273B5F6B0B8>, 'd': <__main__.TrieNode object at 0x00000273B5F6B0F0>, 'm': <__main__.TrieNode object at 0x00000273B5F6BBA8>, 'o': <__main__.TrieNode object at 0x00000273B5F6BF98>, 'g': <__main__.TrieNode object at 0x00000273B68CADA0>, 'k': <__main__.TrieNode object at 0x00000273B68CABE0>, 'p': <__main__.TrieNode object at 0x00000273B68CA7F0>, 'l': <__main__.TrieNode object at 0x00000273B68CA4A8>, 'c': <__main__.TrieNode object at 0x00000273B5BF0BE0>, 'v': <__m

In [23]:
t2.printTrie()

{'m': <__main__.TrieNode object at 0x00000273B6122780>, 'l': <__main__.TrieNode object at 0x00000273B63DF860>, 'w': <__main__.TrieNode object at 0x00000273B63DFC50>, 'f': <__main__.TrieNode object at 0x00000273B63DFB00>, 'e': <__main__.TrieNode object at 0x00000273B63DF940>, 'j': <__main__.TrieNode object at 0x00000273B63DF550>, 'n': <__main__.TrieNode object at 0x00000273B63DF630>, 's': <__main__.TrieNode object at 0x00000273B63DFB38>, 'a': <__main__.TrieNode object at 0x00000273B63DF978>, 'i': <__main__.TrieNode object at 0x00000273B5F6B518>, 'r': <__main__.TrieNode object at 0x00000273B345B400>, 'o': <__main__.TrieNode object at 0x00000273B37C7A58>, 'p': <__main__.TrieNode object at 0x00000273B37C7278>, 'c': <__main__.TrieNode object at 0x00000273B37C7B70>, 'd': <__main__.TrieNode object at 0x00000273B453AC88>, 'u': <__main__.TrieNode object at 0x00000273B364BA58>, 'g': <__main__.TrieNode object at 0x00000273B364B8D0>, 't': <__main__.TrieNode object at 0x00000273B364B780>, 'b': <__m